# Criando Robô Pivot point 

In [1]:
import MetaTrader5 as mt5
from datetime import datetime
import time
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [49]:
# estabelecemos a conexão ao MetaTrader 5
# conecte-se ao MetaTrader 5
if not mt5.initialize():
    print("initialize() Falha ao Iniciar seu metra Trade 5")
    mt5.shutdown()

In [50]:
### Obtendo valores de ativos
#tenho que fazer uma estrutura de repecticao  para ficar gemrado os pontos de entrar e saida
def get_ohlc(ativo, timeframe, n=5):
    ativo = mt5.copy_rates_from_pos(ativo,timeframe, 0, n)
    ativo = pd.DataFrame(ativo)
    ativo['time'] = pd.to_datetime(ativo['time'], unit='s')
    ativo['Pivot'] = (ativo['high'] + ativo['low'] + ativo['close'])/3
    ativo['R1'] = 2*ativo['Pivot'] - ativo['low']
    ativo['S1'] = 2*ativo['Pivot'] - ativo['high']
    ativo['R2'] = ativo['Pivot'] + (ativo['high'] - ativo['low'])
    ativo['S2'] = ativo['Pivot'] - (ativo['high'] - ativo['low'])
    ativo['R3'] = ativo['Pivot'] + 2*(ativo['high'] - ativo['low'])
    ativo['S3'] = ativo['Pivot'] - 2*(ativo['high'] - ativo['low'])

    
    ativo.set_index('time', inplace = True)
    return ativo

In [51]:
ativo = (get_ohlc('ITUB4', mt5.TIMEFRAME_M5))

In [52]:
#verifica se está calculando 
ativo

,open,high,low,close,tick_volume,spread,real_volume,Pivot,R1,S1,R2,S2,R3,S3
time,,,,,,,,,,,,,,
2020-10-26 12:35:00,25.73,25.74,25.70,25.70,122,1,292200,25.713333,25.726667,25.686667,25.753333,25.673333,25.793333,25.633333
2020-10-26 12:40:00,25.70,25.71,25.65,25.68,200,1,218900,25.680000,25.710000,25.650000,25.740000,25.620000,25.800000,25.560000
2020-10-26 12:45:00,25.70,25.71,25.65,25.65,114,1,126900,25.670000,25.690000,25.630000,25.730000,25.610000,25.790000,25.550000
2020-10-26 12:50:00,25.65,25.75,25.65,25.75,368,1,229400,25.716667,25.783333,25.683333,25.816667,25.616667,25.916667,25.516667
2020-10-26 12:55:00,25.75,25.75,25.73,25.74,6,1,2700,25.740000,25.750000,25.730000,25.760000,25.720000,25.780000,25.700000


### preparando a orden

In [53]:
#testando se o ativo é valido 
symbol = 'ITUB4'
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(symbol, "Não encontrato")
    mt5.shutdown()
    quit()

In [54]:
#adicionado symbol se nao existir
if not symbol_info.visible:
    print('Symbol Não visivel, tentnado adicionar')
    if not mt5.symbol_select(symbol,True):
        print('symbol_select({{}})failed, exit', symbol)
        mt5.shutdown()
        quit()

In [55]:
#preparando a ordem compra request  e ordem de venda

#########################################################
#preparando a ordem 
lot = 100.0
point = mt5.symbol_info(symbol).point
price = mt5.symbol_info_tick(symbol).ask
desviation = 1
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY,
    "price": price,
    "magic": 234000,
    "desviation": desviation,
    "comment": "prython script open",
    "type_time":mt5.ORDER_TIME_GTC,
    'type_filling':mt5.ORDER_FILLING_RETURN,
    
          }

In [56]:
#preparando a ordem de venda 
lot = 100.0
point = mt5.symbol_info(symbol).point
price=mt5.symbol_info_tick(symbol).bid
desviation = 1
request2={
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL,
    "price": price,
    
    "deviation": desviation,
    "magic": 234000,
    "comment": "python script close",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_RETURN,
}

In [ ]:
#ENVIO de Ordem buy
result = mt5.order_send(request)
#verificando a resultado da execulção 
print(f'1. Ordem enviada:{lot} de {symbol} ao preço de {price} com desvio de {desviation}',end  = '\r')

In [ ]:
tempo = time.time() + 18000
while time.time() < tempo:
    ativo = (get_ohlc('ITUB4', mt5.TIMEFRAME_M5))
    tick = mt5.symbol_info_tick('ITUB4')
    print (f'ITUB4 - ultimo valor: {tick.last}, Topo do Book C: {tick.bid},Topo do Book V: {tick.ask}', tick.last>ativo['Pivot'][-1 -1], end  = '\r')
    if tick.last> ativo['Pivot'][-1 -1]:
        if mt5.positions_get(symbol="ITUB4") == () or mt5.positions_get(symbol="ITUB4")[0][5] == 1:
            #enviadno ordem de compra 
            result = mt5.order_send(request)
            print(f'1. Ordem COMPRA enviada:{lot} de {symbol} ao preço de {price} com desvio de {desviation}',end  = '\r')

    if tick.last <ativo['S1'][-1 -1]:
        
        if mt5.positions_get(symbol="ITUB4") == () or mt5.positions_get(symbol="ITUB4")[0][5] == 0:
            #enviadno ordem de venda 
            result = mt5.order_send(request2)
        #verificando a resultado da execulção 
            print(f'1. Ordem VENDA enviada: {lot} de {symbol} ao preço de {price} com desvio de {desviation}',end  = '\r')
        
    time.sleep(0.5)